In [1]:
import pandas as pd
import numpy as np
import timeit

In [2]:
#!pip install git+https://github.com/RobustBench/robustbench.git
import robustbench
from robustbench.utils import load_model,clean_accuracy
from robustbench.data import load_cifar10

# Load models and data

In [3]:
import os
#os.chdir('D:\\PFE black\\3.Robustbench and torchattacks')
os.getcwd()

'D:\\PFE black\\3.Robustbench and torchattacks'

In [4]:
x_test, y_test = load_cifar10(n_examples=30)

from robustbench.utils import load_model
model_carmon2019 = load_model(model_name='Carmon2019Unlabeled', threat_model='Linf')

Files already downloaded and verified


In [5]:
model_1 = load_model(model_name='Rebuffi2021Fixing_70_16_cutmix_extra', threat_model='Linf')

In [6]:
n_examples = y_test.shape[0]

In [7]:
#model_2 = load_model(model_name='Gowal2021Improving_70_16_ddpm_100m', threat_model='Linf')

In [8]:
#model_3 = load_model(model_name='Gowal2020Uncovering_70_16_extra', threat_model='Linf')

# torchattacks

In [9]:
#!pip install torchattacks
import torchattacks
import torch

In [10]:
#initialize variables for comparison
name_stats = []
rob_acc_stats = []
time_stats = []

##### Clean accuracy for Carmon2019

In [11]:
acc = clean_accuracy(model_carmon2019, x_test, y_test)
print('Acc carmon2019: %2.2f %%'%(acc*100))

Acc carmon2019: 93.33 %


##### Clean accuracy for model rank1

In [12]:
acc = clean_accuracy(model_1, x_test, y_test)
print('Acc model_1: %2.2f %%'%(acc*100))

Acc model_1: 96.67 %


In [13]:
#choose only corrected classifed images

## Transfer attack PGD

##### source: model rank1
##### target: Carmon2019

In [14]:
attack = torchattacks.PGD(model_1, eps=8/255, alpha=1/255, steps=10, random_start=True)
start_time = timeit.default_timer()
adv_images_PGD = attack(x_test, y_test)
end_time = timeit.default_timer()

In [15]:
acc = clean_accuracy(model_carmon2019, adv_images_PGD, y_test)
print('Robust Acc PGD: %2.2f %%'%(acc*100))

Robust Acc PGD: 76.67 %


## Transfer attack MIFGSM vs DIFGSM

##### source: model rank1
##### target: carmon2019

In [16]:
attack = torchattacks.MIFGSM(model_1, eps=8/255, steps=10, decay=1.0)
start_time = timeit.default_timer()
adv_images_MI = attack(x_test, y_test)
end_time = timeit.default_timer()

In [17]:
acc = clean_accuracy(model_carmon2019, adv_images_MI, y_test)
print('Robust Acc MI: %2.2f %%'%(acc*100))

Robust Acc MI: 76.67 %


In [18]:
attack = torchattacks.DIFGSM(model_1, eps=8/255, alpha=2/255, steps=10, decay=0.0, resize_rate=0.9, diversity_prob=0.5, random_start=False)
start_time = timeit.default_timer()
adv_images_DI = attack(x_test, y_test)
end_time = timeit.default_timer()

In [19]:
acc = clean_accuracy(model_carmon2019, adv_images_DI, y_test)
print('Robust Acc DI: %2.2f %%'%(acc*100))

Robust Acc DI: 73.33 %


##### source: carmon2019 
##### target: model rank1

In [20]:
attack = torchattacks.MIFGSM(model_carmon2019, eps=8/255, steps=10, decay=1.0)
start_time = timeit.default_timer()
adv_images_MI = attack(x_test, y_test)
end_time = timeit.default_timer()

In [21]:
acc = clean_accuracy(model_1, adv_images_MI, y_test)
print('Robust Acc MI: %2.2f %%'%(acc*100))

Robust Acc MI: 83.33 %


In [22]:
attack = torchattacks.DIFGSM(model_carmon2019, eps=8/255, alpha=2/255, steps=10, decay=0.0, resize_rate=0.9, diversity_prob=0.5, random_start=False)
start_time = timeit.default_timer()
adv_images_DI = attack(x_test, y_test)
end_time = timeit.default_timer()

In [23]:
acc = clean_accuracy(model_1, adv_images_DI, y_test)
print('Robust Acc DI: %2.2f %%'%(acc*100))

Robust Acc DI: 83.33 %
